# A Guide to TF Layers: Building a Convolutional Neural Network

I am doing a tutorial from the tensorflow website for CNNs. The url for this tutorial can be found [here](https://www.tensorflow.org/tutorials/layers)

In [1]:
import numpy as np
import random
from datetime import datetime
from sklearn.datasets import load_iris,fetch_mldata
from sklearn.linear_model import Perceptron
from sklearn.metrics import mean_squared_error, accuracy_score, log_loss
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
print("Tensorflow:", tf.__version__)

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here
# if __name__ == "__main__":
#   tf.app.run()

import sys
print("Python Version:", sys.version)

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

(u'Tensorflow:', '1.1.0')
(u'Python Version:', '2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 17:05:42) \n[GCC 7.2.0]')


I am using the estimator API to construct a model function to hold my model. It's a basic 2 layer cnn conv-pool model with a fully conected layer up top (with some dropout). I use ReLu activations for everything except for predictions, which uses softmax. 

If it is in predict mode, I just send back the estimator spec as is. If it's training, I also add the optimizer and training operation in the as well. I am using Adam optimization with a cross entropy loss functionwith an lr of 0.001 with the default alpha (0.9), beta (0.999), and episilon(1e-8).

Finally, if it's eval mode, I add the accuracy onto it (since it's a classifaction problem). 

In [12]:
def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    flat = tf.reshape(pool2, [-1, 7*7*64])
    dense = tf.layers.dense(inputs=flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1), # For PREDICT and EVAL
        "probabilities": tf.nn.softmax(logits, name="softmax")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_op = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_op)
   

Let's load some MNIST data into here. 

In [13]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
test_data = mnist.test.images
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


Now we are going to create the actual estimator!

In [14]:
mnist_clf = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="./cnn_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}


To keep an eye on the training, we are going to attach a logging hook onto it.

In [15]:
tensors_to_log = {"probabilities": "softmax"}
log_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

Let's train this sucker!

In [17]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_clf.train(input_fn=train_input_fn, steps=500, hooks=[log_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./cnn_model/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into ./cnn_model/model.ckpt.
INFO:tensorflow:loss = 2.19398, step = 2
INFO:tensorflow:global_step/sec: 3.63025
INFO:tensorflow:loss = 0.121737, step = 102 (27.548 sec)
INFO:tensorflow:global_step/sec: 3.42828
INFO:tensorflow:loss = 0.102046, step = 202 (29.169 sec)
INFO:tensorflow:global_step/sec: 3.41559
INFO:tensorflow:loss = 0.0784528, step = 302 (29.278 sec)
INFO:tensorflow:global_step/sec: 3.44141
INFO:tensorflow:loss = 0.0832209, step = 402 (29.058 sec)
INFO:tensorflow:probabilities = [[ 0.00000217  0.99898368  0.00009762  0.00000131  0.00081133  0.00000083
   0.00000707  0.00000243  0.00007146  0.00002212]
 [ 0.          0.          0.          0.00027576  0.0000003   0.99964869
   0.00000167  0.00000006  0.00000684  0.00006659]
 [ 0.          0.00000013  0.00000002  0.          0.99999893  0.
   0.00000006  0.00000082  0.       

INFO:tensorflow:Saving checkpoints for 501 into ./cnn_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0340165.


In [19]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=False)
eval_result = mnist_clf.evaluate(input_fn=eval_input_fn)
print(eval_result)

INFO:tensorflow:Starting evaluation at 2018-03-24-19:59:52
INFO:tensorflow:Restoring parameters from ./cnn_model/model.ckpt-501
INFO:tensorflow:Finished evaluation at 2018-03-24-20:00:00
INFO:tensorflow:Saving dict for global step 501: accuracy = 0.9825, global_step = 501, loss = 0.0527337
{'loss': 0.052733701, 'global_step': 501, u'accuracy': 0.98250002}
